In [2]:
import requests

def getTicker(company_name):
    yfinance = "https://query2.finance.yahoo.com/v1/finance/search"
    user_agent = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.0.0 Safari/537.36'
    params = {"q": company_name, "quotes_count": 1, "country": "United States"}

    res = requests.get(url=yfinance, params=params, headers={'User-Agent': user_agent})
    data = res.json()

    company_code = data['quotes'][0]['symbol']
    return company_code

In [3]:
getTicker('Tesla')

'TSLA'

In [4]:
getTicker('Sensex')

'^BSESN'

In [33]:
import yfinance as yf
import pandas as pd

def get_stock_data(company_name, start_date=None, end_date=None):
  """
  This function retrieves historical stock price data for a company (checks availability, avoids downloads).

  Args:
      company_name (str): The name of the company.
      start_date (str, optional): The starting date in YYYY-MM-DD format. Defaults to None (maximum available).
      end_date (str, optional): The ending date in YYYY-MM-DD format. Defaults to None (today).

  Returns:
      pandas.DataFrame: A DataFrame containing historical stock prices (empty if unavailable).
  """

  # Get ticker symbol (assuming you have a separate function for this)
  ticker_symbol = getTicker(company_name)

  # Create a yfinance Ticker object
  ticker = yf.Ticker(ticker_symbol)

  # Download historical data for the specified period (avoid downloading all)
  if start_date and end_date:
      data = ticker.history(start=start_date, end=end_date)
  else:
      sensex = getTicker('Sensex')
      ticker_default = yf.Ticker(sensex)
      data = ticker_default.history(period = '3mo')
    
  # Check if DataFrame is empty (handle unavailable company data)
  if data.empty:
      print('Sensex!')
      sensex = getTicker('Sensex')
      ticker_default = yf.Ticker(sensex)
      data = ticker_default.history(start = start_date,end = end_date)
  else:
      pass

  # No data returned, but avoid downloading Sensex (empty DataFrame)
  return data

In [41]:
company_name = 'Microsoft'
start_date = "2020-01-01"
end_date = "2024-05-22"

stock_data = get_stock_data(company_name, start_date, end_date)

In [42]:
stock_data.reset_index(inplace=True)
stock_data['Date'] = stock_data['Date'].dt.strftime('%Y-%m-%d')
stock_data.drop(['Dividends','Stock Splits'], inplace=True, axis=1)
stock_data.to_dict(orient='records')



[{'Date': '2020-01-02',
  'Open': 152.44902163274443,
  'High': 154.32126706184013,
  'Low': 152.01696724916556,
  'Close': 154.2156524658203,
  'Volume': 22622100},
 {'Date': '2020-01-03',
  'Open': 152.00738364396642,
  'High': 153.57238141210834,
  'Low': 151.75774113281116,
  'Close': 152.29541015625,
  'Volume': 21116200},
 {'Date': '2020-01-06',
  'Open': 150.8168410749267,
  'High': 152.75630287642375,
  'Low': 150.2695613255879,
  'Close': 152.6890869140625,
  'Volume': 20813700},
 {'Date': '2020-01-07',
  'Open': 152.9675327776384,
  'High': 153.30356863974853,
  'Low': 151.04727762140624,
  'Close': 151.29690551757812,
  'Volume': 21634100},
 {'Date': '2020-01-08',
  'Open': 152.5930509925824,
  'High': 154.38849931453737,
  'Low': 151.65213017891352,
  'Close': 153.70680236816406,
  'Volume': 27746500},
 {'Date': '2020-01-09',
  'Open': 155.38698236042114,
  'High': 155.75183538413958,
  'Low': 154.60928167333998,
  'Close': 155.62701416015625,
  'Volume': 21385000},
 {'Date

In [43]:
import json


def analyze_stock_data(stock_data):
  """
  Analyzes stock data and returns a dictionary with key metrics.

  Args:
      stock_data: A list of dictionaries, where each dictionary represents
          a day's stock data with keys 'Date', 'Open', 'High', 'Low',
          'Close', and 'Volume'.

  Returns:
      A dictionary containing the following key metrics:
          - greatest_increase: The date and percentage increase of the
              greatest increase in the last month.
          - greatest_decrease: The date and percentage decrease of the
              greatest decrease in the last month.
          - current_close: The closing price of the last day.
          - volatility: The average daily fluctuation over the last week.
          - moving_average_1_month: The 30-day moving average of the closing price.
  """

  # Convert data to pandas DataFrame for easier manipulation
  
  df = pd.DataFrame(stock_data)

  # Ensure dates are parsed as datetime
  df['Date'] = pd.to_datetime(df['Date'])

  # Filter data for the last month
  last_month_data = df[df['Date'].dt.month == df['Date'].iloc[-1].month]

  # Greatest increase
  greatest_increase = last_month_data.sort_values(by='Close', ascending=False).iloc[0]
  greatest_increase_pct = (greatest_increase['Close'] - greatest_increase['Open']) / greatest_increase['Open'] * 100

  # Greatest decrease
  greatest_decrease = last_month_data.sort_values(by='Close', ascending=True).iloc[0]
  greatest_decrease_pct = (greatest_decrease['Open'] - greatest_decrease['Close']) / greatest_decrease['Open'] * 100

  # Current close value
  current_close = df.iloc[-1]['Close']

  # Volatility (average daily fluctuation over the last week)
  last_week_data = df[df['Date'] >= df['Date'].iloc[-1] - pd.Timedelta(days=7)]
  volatility = last_week_data['Close'].pct_change().std() * 100

  # Moving average of the stock over 1 month
  moving_average_1_month = df['Close'].rolling(window=30).mean().iloc[-1]

  # Create and return the dictionary
  results = {
      "greatest_increase": {
          "date": greatest_increase['Date'].strftime('%Y-%m-%d'),
          "increase_pct": greatest_increase_pct
      },
      "greatest_decrease": {
          "date": greatest_decrease['Date'].strftime('%Y-%m-%d'),
          "decrease_pct": greatest_decrease_pct
      },
      "current_close": current_close,
      "volatility": volatility,
      "moving_average_1_month": moving_average_1_month
  }

  return json.dumps(results)




analysis_results = analyze_stock_data(stock_data)
print(analysis_results)


{"greatest_increase": {"date": "2024-05-21", "increase_pct": 0.5177757051242406}, "greatest_decrease": {"date": "2020-05-01", "decrease_pct": 0.6996562606298389}, "current_close": 429.0400085449219, "volatility": 0.946763134737521, "moving_average_1_month": 410.7175587972005}
